In [5]:
#!pip install --upgrade celery

In [6]:
import celery
print(celery.__version__)
#5.1.2

5.1.2


In [7]:
!sudo rabbitmqctl add_user myguest myguestpwd
!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

Creating user "myguest" ...
Error: user_already_exists: myguest
Setting permissions for user "myguest" in vhost "/" ...


In [8]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a **test.py** file. <br> 
You will need to run that code on all of your **worker** machines with <br>

<code>celery -A test worker --loglevel=info" </code>

Then that machine becomes a worker and will be able to run the "test" application (i.e. in this case the echo function in test.py) whenever the broker requests it.

The echo function will be called by these commands on the remote (=worker) machines:

```python
from test import echo  
res = echo.delay('Python rocks!')  
res.result <br>
```

In [10]:
from test import echo

Before you run the next cell, you will need to <br>
<code> pip install celery </code> on the worker machine and then run <br>
<code> celery -A test worker --loglevel=info <code> on that machine.



In [11]:
# We will run the echo function on the worker machine.
# The fun.delay(arg) function sends the arg message to the worker's fun function and run it on the worker machine.
# For more info, see https://docs.celeryproject.org/en/latest/userguide/calling.html
    
# This code will send 'Python rocks 1 :) !' to the worker machine, and put its response into res1
res1 = echo.delay('Python rocks 1 :) !')

print(type(res1))
print(res1)

<class 'celery.result.AsyncResult'>
df808041-c479-4073-91bd-82c017251952


In [12]:
# You will need backend to be able to get the below results.
# As you can see in the test.py we are using 'rpc://...' backend.
res1.ready()

True

In [13]:
res1.result

'Response from worker: Python rocks 1 :) !'

In [14]:
# Let us repeat it with a different message: 
# Tasks will be distributed round robin among the workers

res2 = echo.delay('Python rocks 2 :) !'); print(type(res2)); print(res2)
res2.ready()

<class 'celery.result.AsyncResult'>
b9873a16-8ff0-4612-9eb7-993f568e41a3


False

As you can see res2 is asyncronous and might not be immedialtely available.

In [15]:
print(res2.result)

Response from worker: Python rocks 2 :) !


In [16]:
res2.ready()

True

Note that by default only **one** of the workers will work on your 'echo' task.
If you call this task multiple times, then the tasks will be distributed **round robin** among the workers.
In the below experiment I used 3 aws instances as workers.

In [22]:
results=[]
for it in range(10):
    res = echo.delay(f'Python rocks {it} :) !')
    results.append(res)
    print(f'{it}: {res}, {res.ready()}, {res.result}')

0: 824c6271-88ce-4e82-ba77-7c51ccc719a7, False, Response from worker: Python rocks 0 :) !
1: 0215e58d-c955-4069-8542-ff55ec429071, False, Response from worker: Python rocks 1 :) !
2: 69dc4c13-1911-42fa-b826-4d12cc9042b7, False, None
3: 726432c5-535c-4717-8456-4b661960b623, False, Response from worker: Python rocks 3 :) !
4: 608513bb-3369-403a-af3d-47233be969d1, False, Response from worker: Python rocks 4 :) !
5: 4b8fd96e-7ff5-4528-8adf-478b491f664e, False, Response from worker: Python rocks 5 :) !
6: 0888780c-fa04-4d71-a211-3bdbcd96c247, False, Response from worker: Python rocks 6 :) !
7: b9993f0e-dd73-44f3-9178-e13c9783fb88, False, Response from worker: Python rocks 7 :) !
8: 2ebadac2-c89d-449a-9f60-61c99a0ae7de, False, None
9: 8c25b976-9bf3-4a4f-b978-fc3d5efa6648, False, Response from worker: Python rocks 9 :) !


As you can see above, some of the results are avaialble almost immediately, for the other we will have to wait.
Below I show that soon all results become available.

In [23]:
for it in range(10):
    res=results[it]
    print(f'{it}: {res}, {res.ready()}, {res.result}')

0: 824c6271-88ce-4e82-ba77-7c51ccc719a7, True, Response from worker: Python rocks 0 :) !
1: 0215e58d-c955-4069-8542-ff55ec429071, True, Response from worker: Python rocks 1 :) !
2: 69dc4c13-1911-42fa-b826-4d12cc9042b7, True, Response from worker: Python rocks 2 :) !
3: 726432c5-535c-4717-8456-4b661960b623, True, Response from worker: Python rocks 3 :) !
4: 608513bb-3369-403a-af3d-47233be969d1, True, Response from worker: Python rocks 4 :) !
5: 4b8fd96e-7ff5-4528-8adf-478b491f664e, True, Response from worker: Python rocks 5 :) !
6: 0888780c-fa04-4d71-a211-3bdbcd96c247, True, Response from worker: Python rocks 6 :) !
7: b9993f0e-dd73-44f3-9178-e13c9783fb88, True, Response from worker: Python rocks 7 :) !
8: 2ebadac2-c89d-449a-9f60-61c99a0ae7de, True, Response from worker: Python rocks 8 :) !
9: 8c25b976-9bf3-4a4f-b978-fc3d5efa6648, True, Response from worker: Python rocks 9 :) !


In [18]:
echo('hello')

'Response from worker: hello'